In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


In [16]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [17]:
train = pd.read_csv('../../dataset/train.csv')
valid = pd.read_csv('../../dataset/val.csv')
test = pd.read_csv('../../dataset/test.csv')

In [18]:
train

,Emotion,Comment
0,0,On days when I feel close to my partner and ot...
1,1,Every time I imagine that someone I love or I ...
2,2,When I had been obviously unjustly treated and...
3,3,When I think about the short time that we live...
4,4,At a gathering I found myself involuntarily si...
...,...,...
5179,1,My parents were out and I was the eldest at ho...
5180,5,Two years back someone invited me to be the tu...
5181,5,I had taken the responsibility to do something...
5182,1,I was at home and I heard a loud sound of spit...


In [19]:
#IMP DATA FOR CONFIG

AUTO = tf.data.experimental.AUTOTUNE


# Configuration
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 168

In [20]:
xtrain, ytrain = train.Comment.values, train.Emotion.values
xvalid, yvalid = valid.Comment.values, valid.Emotion.values
xtest, ytest = test.Comment.values, test.Emotion.values
                                                  

train['Comment'].apply(lambda x:len(str(x).split())).max()

168

In [21]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 168

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)
xtest_seq = token.texts_to_sequences(xtest)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)
xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)

word_index = token.word_index

In [22]:
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [23]:
with strategy.scope():
    # A simpleRNN without any pretrained embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len))
    model.add(LSTM(128))
    model.add(Dense(7, activation='softmax'))
    model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])
    
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 168, 300)          4167000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 903       
Total params: 4,387,551
Trainable params: 4,387,551
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(xtrain_pad, ytrain,epochs=10,validation_data=(xvalid_pad, yvalid), batch_size=64*strategy.num_replicas_in_sync) #Multiplying by Strategy to run on TPU's

Epoch 1/10
81/81 [==============================] - 24s 302ms/step - loss: 1.9245 - accuracy: 0.1966 - val_loss: 1.9172 - val_accuracy: 0.2255
Epoch 2/10
81/81 [==============================] - 26s 317ms/step - loss: 1.7239 - accuracy: 0.4641 - val_loss: 1.8540 - val_accuracy: 0.2809
Epoch 3/10
81/81 [==============================] - 25s 312ms/step - loss: 1.5380 - accuracy: 0.6584 - val_loss: 1.8133 - val_accuracy: 0.3047
Epoch 4/10
81/81 [==============================] - 27s 328ms/step - loss: 1.4247 - accuracy: 0.7552 - val_loss: 1.8100 - val_accuracy: 0.3209
Epoch 5/10
81/81 [==============================] - 26s 322ms/step - loss: 1.3629 - accuracy: 0.8167 - val_loss: 1.8058 - val_accuracy: 0.3209
Epoch 6/10
81/81 [==============================] - 24s 296ms/step - loss: 1.3315 - accuracy: 0.8439 - val_loss: 1.8166 - val_accuracy: 0.3115
Epoch 7/10
81/81 [==============================] - 23s 284ms/step - loss: 1.3004 - accuracy: 0.8708 - val_loss: 1.8085 - val_accuracy: 0.3243

In [14]:
model.evaluate(xtest_pad, ytest)

37/37 [==============================] - 2s 57ms/step - loss: 1.8464 - accuracy: 0.2835


[1.846409559249878, 0.2834917902946472]